# In-memory logging for realtime applications

 DPsim integrates a datalogger for realtime applications. It pre-allocates memory to run without writing to a file until the simulation is finished.

 For normal simulations it also increases the speed of the execution.

## Example 1

We start by defining a very simple simulation:

In [ ]:
import dpsimpy

# Parameters
duration = 1
timestep = 0.001
name = "ExampleDatalogger1"
log_path = "./logs/" + name + ".csv"
attributes = 4  # real and imaginary for a voltage and a currrent

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode("n1")

# Components
v1 = dpsimpy.dp.ph1.VoltageSource("v_1")
v1.set_parameters(V_ref=complex(345, 0), f_src=50)
v1.connect([gnd, n1])
r1 = dpsimpy.dp.ph1.Resistor("r1")
r1.connect([gnd, n1])

sys = dpsimpy.SystemTopology(50, [gnd, n1], [v1, r1])

We setup now a Realtime Datalogger

In [ ]:
logger = dpsimpy.RealTimeDataLogger(log_path, duration, timestep)
logger.log_attribute("n1.v", "v", n1)
logger.log_attribute("r1.i", "i_intf", r1)

Now we setup a simple simulation

In [ ]:
sim = dpsimpy.RealTimeSimulation(name, dpsimpy.LogLevel.debug)
sim.set_system(sys)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_time_step(timestep)
sim.set_final_time(duration)
sim.add_logger(logger)
sim.run(5)

Check that all went good

In [ ]:
import pandas as pd

# Read the CSV
df = pd.read_csv(log_path)

# Expected dimensions
expected_rows = int(duration / timestep + 0.5)  # timesteps + 1
expected_cols = 1 + attributes  # time + attributes

# Assertions
assert df.shape[0] == expected_rows, f"Expected {expected_rows} rows, got {df.shape[0]}"
assert (
    df.shape[1] == expected_cols
), f"Expected {expected_cols} columns, got {df.shape[1]}"

print("OK:", df.shape)